In [1]:
## Importar librerías
import pandas as pd
import sqlite3

# Cargar base de datos
Queremos ver que tablas contiene para poder empezar la investigación.

In [2]:
### Conexión al archivo .db
con = sqlite3.connect("sql-murder-mystery.db")
query = "SELECT name FROM sqlite_master WHERE type='table';"
tablas = pd.read_sql(query,con)
tablas

,name
0,crime_scene_report
1,drivers_license
2,person
3,facebook_event_checkin
4,interview
5,get_fit_now_member
6,get_fit_now_check_in
7,income
8,solution


## Info inicial
Sabemos que ocurrió el 15 de enero del 2018, en la ciudad de SQL City.

In [3]:
res = "SELECT * FROM crime_scene_report WHERE date = 20180115 AND city = 'SQL City'"
res2 = pd.read_sql(res,con)
res2

,date,type,description,city
0,20180115,assault,"Hamilton: Lee, do you yield? Burr: You shot hi...",SQL City
1,20180115,assault,Report Not Found,SQL City
2,20180115,murder,Security footage shows that there were 2 witne...,SQL City


Estamos buscando un asesinato, por lo que descartamos los primeros dos registros y miramos el tercero.

In [4]:
res2.iloc[2,2]

'Security footage shows that there were 2 witnesses. The first witness lives at the last house on "Northwestern Dr". The second witness, named Annabel, lives somewhere on "Franklin Ave".'

Sabemos que el primer testigo vive en la última casa de **Northwestern Dr**. El segundo se llama **Annabel** y vive en alguna casa de **Franklin Ave** .

## Primer testigo
Analizamos la estructura de la tabla de personas

In [5]:
res = "SELECT * FROM person"
res2 = pd.read_sql(res,con)
res2

,id,name,license_id,address_number,address_street_name,ssn
0,10000,Christoper Peteuil,993845,624,Bankhall Ave,747714076
1,10007,Kourtney Calderwood,861794,2791,Gustavus Blvd,477972044
2,10010,Muoi Cary,385336,741,Northwestern Dr,828638512
3,10016,Era Moselle,431897,1987,Wood Glade St,614621061
4,10025,Trena Hornby,550890,276,Daws Hill Way,223877684
...,...,...,...,...,...,...
10006,99936,Luba Benser,274427,680,Carnage Blvd,685095054
10007,99941,Roxana Mckimley,975942,1613,Gate St,512136801
10008,99965,Cherie Zeimantz,287627,3661,The Water Ave,362877324
10009,99982,Allen Cruse,251350,3126,N Jean Dr,348734531


Filtramos la dirección y ordenamos de manera descendente el número de casas, para obtener la última.

In [6]:
witness_1 = "SELECT * FROM person WHERE address_street_name = 'Northwestern Dr' ORDER BY address_number DESC LIMIT 1"
witness_1f = pd.read_sql(witness_1,con)
witness_1f

,id,name,license_id,address_number,address_street_name,ssn
0,14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949


## Segundo testigo
Filtramos la dirección y el nombre que sea Annabel.

In [8]:
witness_2 = "SELECT * FROM person WHERE address_street_name = 'Franklin Ave' AND name LIKE 'Annabel%'"
witness_2f = pd.read_sql(witness_2,con)
witness_2f

,id,name,license_id,address_number,address_street_name,ssn
0,16371,Annabel Miller,490173,103,Franklin Ave,318771143


Revisamos la tabla de interviews para ver como podemos accesar la información de cada individuo.

In [9]:
entrevistas = "SELECT * FROM interview"
entrevistas2 = pd.read_sql(entrevistas,con)
entrevistas2

,person_id,transcript
0,28508,‘I deny it!’ said the March Hare.\n
1,63713,\n
2,86208,"way, and the whole party swam to the shore.\n"
3,35267,"lessons in here? Why, there’s hardly room for ..."
4,33856,\n
...,...,...
4986,37357,Alice did not wish to offend the Dormouse agai...
4987,10206,"time,’ she said, ‘than waste it in asking ridd..."
4988,14887,I heard a gunshot and then saw a man run out. ...
4989,16371,"I saw the murder happen, and I recognized the ..."


## Entrevista
Con el id de los testigos, podremos ver el testimonio que dieron a la policía.

In [10]:
res = "SELECT * FROM interview WHERE person_id = 14887 OR person_id = 16371"
res2 = pd.read_sql(res,con)
res2

,person_id,transcript
0,14887,I heard a gunshot and then saw a man run out. ...
1,16371,"I saw the murder happen, and I recognized the ..."


In [11]:
# Morty Schapiro
res2.iloc[0,1]

'I heard a gunshot and then saw a man run out. He had a "Get Fit Now Gym" bag. The membership number on the bag started with "48Z". Only gold members have those bags. The man got into a car with a plate that included "H42W".'

In [12]:
#Annabel Miller
res2.iloc[1,1]

'I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th.'

## Perfil de sospechoso
Sexo **Masculino**, visita el **gym**, tiene membresía **gold** con el número **48z** y se subió a un vehículo con placa **H42W**

In [13]:
res = "SELECT * FROM get_fit_now_member"
res2 = pd.read_sql(res,con)
res2

,id,person_id,name,membership_start_date,membership_status
0,NL318,65076,Everette Koepke,20170926,gold
1,AOE21,39426,Noe Locascio,20171005,regular
2,2PN28,63823,Jeromy Heitschmidt,20180215,silver
3,0YJ24,80651,Waneta Wellard,20171206,gold
4,3A08L,32858,Mei Bianchin,20170401,silver
...,...,...,...,...,...
179,2V137,41693,Wendell Dulany,20171219,silver
180,4KB72,79110,Emile Hege,20170522,regular
181,48Z7A,28819,Joe Germuska,20160305,gold
182,48Z55,67318,Jeremy Bowers,20160101,gold


In [14]:
res = "SELECT * FROM get_fit_now_member WHERE id LIKE '%48Z%'"
res2 = pd.read_sql(res,con)
res2

,id,person_id,name,membership_start_date,membership_status
0,48Z38,49550,Tomas Baisley,20170203,silver
1,48Z7A,28819,Joe Germuska,20160305,gold
2,48Z55,67318,Jeremy Bowers,20160101,gold


## 2 Sospechosos
Descartamos a Tomas Baisley ya que su membresía no es dorada. Tenemos a **Joe Germuska** y a **Jeremy Bowers** como candidatos. Tendremos que revisar quien tiene la placa del carro indicada y que haya tenido actividad durante la fecha establecida por la testigo Annabel.

In [15]:
res = "SELECT name, license_id FROM person WHERE name = 'Joe Germuska' OR name = 'Jeremy Bowers'"
res2 = pd.read_sql(res,con)
res2

,name,license_id
0,Joe Germuska,173289
1,Jeremy Bowers,423327


In [16]:
res = "SELECT * FROM drivers_license"
res2 = pd.read_sql(res,con)
res2

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,100280,72,57,brown,red,male,P24L4U,Acura,MDX
1,100460,63,72,brown,brown,female,XF02T6,Cadillac,SRX
2,101029,62,74,green,green,female,VKY5KR,Scion,xB
3,101198,43,54,amber,brown,female,Y5NZ08,Nissan,Rogue
4,101255,18,79,blue,grey,female,5162Z1,Lexus,GS
...,...,...,...,...,...,...,...,...,...
10002,999923,19,77,amber,black,female,5L0ZI4,GMC,Sierra 3500
10003,999940,71,61,green,green,male,1B8QN8,Mitsubishi,Eclipse
10004,999981,67,69,brown,blue,female,1684K3,Land Rover,LR2
10005,999986,49,58,green,grey,male,F8F64H,Lexus,LS


In [ ]:
carro = "SELECT * FROM drivers_license WHERE plate_number LIKE '%H42W%'"
carro_f = pd.read_sql(carro,con)
carro_f

## Sospechoso: Jeremy Bowers
Su licencia coincide con la descripción de la placa del carro.

In [17]:
ingreso = "SELECT * FROM get_fit_now_check_in"
ingreso_f = pd.read_sql(ingreso,con)
ingreso_f

,membership_id,check_in_date,check_in_time,check_out_time
0,NL318,20180212,329,365
1,NL318,20170811,469,920
2,NL318,20180429,506,554
3,NL318,20180128,124,759
4,NL318,20171027,418,1019
...,...,...,...,...
2698,4KB72,20170422,1016,1114
2699,4KB72,20170630,408,885
2700,48Z7A,20180109,1600,1730
2701,48Z55,20180109,1530,1700


In [18]:
ingreso = "SELECT * FROM get_fit_now_check_in WHERE check_in_date BETWEEN 20180109 AND 20180115 AND membership_id = '48Z7A'"
ingreso_f = pd.read_sql(ingreso,con)
ingreso_f

,membership_id,check_in_date,check_in_time,check_out_time
0,48Z7A,20180109,1600,1730


## Confirmación de datos
Podemos corroborar que toda la evidencia apunta a Jeremy Bowers, desde la placa de su carro, hasta su ID, fecha de visita al gimnasio junto con el ID de su membresía y tipo de membresía Gold. Revisaremos su testimonio.

In [19]:
entrevista_jb = "SELECT * FROM interview WHERE person_id = 67318"
entrevista_resultado= pd.read_sql(entrevista_jb,con)
entrevista_resultado

,person_id,transcript
0,67318,I was hired by a woman with a lot of money. I ...


In [20]:
entrevista_resultado.iloc[0,1]

'I was hired by a woman with a lot of money. I don\'t know her name but I know she\'s around 5\'5" (65") or 5\'7" (67"). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017.\n'

## Mastermind Sospechosa
Logramos encontrar al asesino, pero ahora tenemos que encontrar a la mujer encargada de todo el plan. Tenemos su estatura que es de entre **165 cm y 167 cm**, tiene pelo de color **rojo** y maneja un carro **Tesla Model S**. Finalmente sabemos que visitó el **SQL Concert 3 times in December 2017.***

In [21]:
suspect = "SELECT * FROM drivers_license WHERE (height BETWEEN 65 AND 67) AND hair_color = 'red' AND car_make = 'Tesla'"
suspect2 = pd.read_sql(suspect,con)
suspect2

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,202298,68,66,green,red,female,500123,Tesla,Model S
1,291182,65,66,blue,red,female,08CM64,Tesla,Model S
2,918773,48,65,black,red,female,917UU3,Tesla,Model S


## 3 Candidatas
Revisaremos por medio de sus ID's de licencia, el ID de persona y revisar en los eventos de facebook quien haya visitado la orquesta sinfónica.

In [22]:
perfil = "SELECT * FROM person WHERE license_id = 202298 OR license_id = 291182 OR license_id = 918773"
perfil_f = pd.read_sql(perfil,con)
perfil_f

,id,name,license_id,address_number,address_street_name,ssn
0,78881,Red Korb,918773,107,Camerata Dr,961388910
1,90700,Regina George,291182,332,Maple Ave,337169072
2,99716,Miranda Priestly,202298,1883,Golden Ave,987756388


## ID de sospechosas y sus nombres
Tenemos a **Red Korb** con  **ID 78881**, **Regina George** con **ID 90700** y **Miranda Priestly** con **ID 99716**.

In [23]:
perfil = "SELECT * FROM facebook_event_checkin WHERE person_id = 78881 OR person_id = 90700 OR person_id = 99716"
perfil_f = pd.read_sql(perfil,con)
perfil_f

,person_id,event_id,event_name,date
0,99716,1143,SQL Symphony Concert,20171206
1,99716,1143,SQL Symphony Concert,20171212
2,99716,1143,SQL Symphony Concert,20171229


# Veredicto
Hemos resuelto el caso, el asesino es **Jeremy Bowers** y la persona que lo contrató y planeó el homicidio es **Miranda Priestly**. 

In [26]:
cursor = con.cursor()
sql = "INSERT INTO solution VALUES (1, 'Miranda Priestly')"
cursor.execute(sql)
con.commit()
var = pd.read_sql("SELECT value FROM solution", con)
var.iloc[0,0]

'Congrats, you found the brains behind the murder! Everyone in SQL City hails you as the greatest SQL detective of all time. Time to break out the champagne!'